# Predicción de Lluvia en Australia

### Facultad de Ciencias Exactas, Ingeniería y Agrimensura  
### Tecnicatura en Inteligencia Artificial  
### Aprendizaje Automático 1  

**Trabajo Práctico**: Predicción de lluvia utilizando modelos de clasificación y redes neuronales.

**Autores**:  
- Calcia Franco  
- Avecilla Tomas  

**Fecha**: Octubre 2024  

---

### Objetivo
Desarrollar un modelo de predicción de la variable `RainTomorrow` basado en datos climáticos de Australia, utilizando técnicas de pre-procesamiento, modelado, evaluación y optimización de modelos, para su posterior puesta en producción.



### Diccionario de Datos

| Variable         | Descripción                                                       | Tipo        | Ejemplo       | Valores Faltantes |
|-------------------|------------------------------------------------------------------|-------------|---------------|-------------------|
| **Date**          | Fecha de la observación del clima.                               | object      | 2010-01-01    | 0                 |
| **Location**      | Ciudad donde se realizó la medición del clima.                  | object      | Sydney        | 0                 |
| **MinTemp**       | Temperatura mínima del día (en grados Celsius).                 | float64     | 12.3          | 1485              |
| **MaxTemp**       | Temperatura máxima del día (en grados Celsius).                 | float64     | 28.9          | 1261              |
| **Rainfall**      | Cantidad de lluvia caída en milímetros durante el día.          | float64     | 0.0           | 3261              |
| **Evaporation**   | Evaporación en milímetros durante el día.                       | float64     | 0.0           | 62790             |
| **Sunshine**      | Horas de sol durante el día.                                    | float64     | 8.0           | 69835             |
| **WindGustDir**   | Dirección del viento racheado (en grados).                      | object      | NNE           | 10326             |
| **WindGustSpeed** | Velocidad del viento racheado (en km/h).                        | float64     | 15.0          | 10263             |
| **WindDir9am**    | Dirección del viento a las 9 am.                                | object      | NNE           | 10566             |
| **WindDir3pm**    | Dirección del viento a las 3 pm.                                | object      | SSE           | 4228              |
| **WindSpeed9am**  | Velocidad del viento a las 9 am (en km/h).                     | float64     | 5.0           | 1767              |
| **WindSpeed3pm**  | Velocidad del viento a las 3 pm (en km/h).                     | float64     | 10.0          | 3062              |
| **Humidity9am**   | Humedad relativa a las 9 am (%).                               | float64     | 65.0          | 2654              |
| **Humidity3pm**   | Humedad relativa a las 3 pm (%).                               | float64     | 55.0          | 4507              |
| **Pressure9am**   | Presión atmosférica a las 9 am (hPa).                          | float64     | 1012.0        | 15065             |
| **Pressure3pm**   | Presión atmosférica a las 3 pm (hPa).                          | float64     | 1011.0        | 15028             |
| **Cloud9am**      | Cobertura nubosa a las 9 am (en octavos).                      | float64     | 2.0           | 55888             |
| **Cloud3pm**      | Cobertura nubosa a las 3 pm (en octavos).                      | float64     | 3.0           | 59358             |
| **Temp9am**       | Temperatura a las 9 am (en grados Celsius).                    | float64     | 15.0          | 1767              |
| **Temp3pm**       | Temperatura a las 3 pm (en grados Celsius).                    | float64     | 20.0          | 3609              |
| **RainToday**     | Indica si llovió hoy (Yes/No).                                 | object      | Yes           | 3261              |
| **RainTomorrow**  | Indica si lloverá mañana (Yes/No).                             | object      | No            | 3267              |


## Preparación del entorno de trabajo

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [1]:
!gdown "1yBz9nW8bcB7WCiwWhrjaU-Oifl-9L76D" --output "weatherAUS.csv"

Downloading...
From: https://drive.google.com/uc?id=1yBz9nW8bcB7WCiwWhrjaU-Oifl-9L76D
To: c:\Users\Usuario\Desktop\TUIA\AA1-TUIA-Avecilla-Calcia\AA1-TUIA-Calcia-Avecilla\weatherAUS.csv

  0%|          | 0.00/14.1M [00:00<?, ?B/s]
  4%|▎         | 524k/14.1M [00:00<00:03, 3.93MB/s]
 11%|█         | 1.57M/14.1M [00:00<00:02, 4.82MB/s]
 19%|█▊        | 2.62M/14.1M [00:00<00:02, 5.03MB/s]
 26%|██▌       | 3.67M/14.1M [00:00<00:01, 5.29MB/s]
 30%|██▉       | 4.19M/14.1M [00:00<00:01, 5.27MB/s]
 33%|███▎      | 4.72M/14.1M [00:00<00:01, 5.26MB/s]
 41%|████      | 5.77M/14.1M [00:01<00:01, 5.32MB/s]
 48%|████▊     | 6.82M/14.1M [00:01<00:01, 5.44MB/s]
 56%|█████▌    | 7.86M/14.1M [00:01<00:01, 5.51MB/s]
 63%|██████▎   | 8.91M/14.1M [00:01<00:00, 5.41MB/s]
 71%|███████   | 9.96M/14.1M [00:01<00:00, 5.49MB/s]
 78%|███████▊  | 11.0M/14.1M [00:02<00:00, 5.47MB/s]
 86%|████████▌ | 12.1M/14.1M [00:02<00:00, 5.83MB/s]
 93%|█████████▎| 13.1M/14.1M [00:02<00:00, 5.77MB/s]
100%|██████████| 14.1M/14.1M 

In [6]:
df_clima = pd.read_csv("weatherAUS.csv")
print(df_clima.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

## Aalisis Descriptivo

### Analisis de variables en crudo

### Manejo de valores faltantes/nulos

In [4]:
df_clima.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

Como se puede ver tenemos muchas observaciones con datos faltantes por lo cual vamos a trabajar sobre ellos...

### Analisis de variables post limpieza